## ひろしまQuest2020#stayhome【球種予測部門】
#### all_merge_18
- コースの種類、1球前のボール、ファウル数

#### all_merge_17
- イニング時間、打席時間、平均投球間隔(差)、サヨナラ、延長戦

#### all_merge_16
- 球団ごとの年棒ランク
- 選手IDごとの2017年のコース実績

#### all_merge_15
- 左右で球種/コースを別に集計

#### all_merge_14
- 左右で球種を別に集計

#### all_merge_13
- 前の投球からの時間差

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

### 管理番号

In [2]:
model_No = 18

### sub-modelを使用するかどうか

In [3]:
use_sub_model = True

In [4]:
ALL_MERGE = 'intermediate/all_merge_{}.f'.format(model_No)
SUBMIT = 'submit/ball_{}.csv'.format(model_No)

if use_sub_model:
    ALL_MERGE = 'intermediate/all_merge_{}_sub.f'.format(model_No)
    SUBMIT = 'submit/ball_{}_sub.csv'.format(model_No)

OUT_SUBMODEL = 'intermediate/ball_predict_{}.f'.format(model_No)

In [5]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 238)

In [6]:
if use_sub_model:
    all_pitch.drop(columns=[
        'predict_straight',
        'predict_curve',
        'predict_slider',
        'predict_shoot',
        'predict_fork',
        'predict_changeup',
        'predict_sinker',
        'predict_cutball'
    ], inplace=True)

In [7]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 230)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_course00,bc_course01,bc_course02,bc_course03,bc_course04,bc_course05,bc_course06,bc_course07,bc_course08,bc_course09,bc_course10,bc_course11,bc_course12,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,...,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9,predict_10,predict_11,predict_12
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,...,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,0.086459,0.105353,0.064427,0.058526,0.095456,0.067057,0.044388,0.059785,0.046089,0.136055,0.068114,0.069584,0.098707
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.442351,0.093302,0.109348,0.078518,0.141711,0.065447,0.017669,0.051654,0.044893,0.071306,0.068422,0.032002,0.043180,0.036149,0.029388,0.042910,0.033625,0.152078,0.078157,0.203642,0.164248,0.106283,0.230235,0.157397,0.138195,0.367890,0.184621,0.355720,0.111331,0.105923,...,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,0.048093,0.069573,0.054361,0.041596,0.063309,0.047175,0.037439,0.087601,0.043582,0.118144,0.096016,0.135095,0.158017
2,2,0.0,5.0,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.504212,0.069963,0.065568,0.046520,0.211722,0.059158,0.019231,0.023626,0.027656,0.045421,0.045238,0.025275,0.023810,0.027106,0.023077,0.032784,0.022711,0.202930,0.081319,0.285165,0.157509,0.076007,0.284249,0.102015,0.095055,0.442674,0.118315,0.488095,0.076190,0.078571,...,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,0.035639,0.046519,0.037898,0.031871,0.038175,0.042324,0.030407,0.054304,0.035978,0.164639,0.106219,0.217382,0.158646
3,3,0.0,12.0,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.475961,0.101792,0.226048,0.066617,0.042131,0.018345,0.003343,0.065762,0.034319,0.043647,0.027090,0.054336,0.065374,0.045046,0.062653,0.097944,0.085507,0.098410,0.098527,0.055307,0.231840,0.151308,0.196937,0.206965,0.157643,0.287147,0.105057,0.153718,0.164756,0.246104,...,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1,0.041884,0.032665,0.015660,0.093557,0.092507,0.044834,0.096822,0.117248,0.092368,0.112900,0.096562,0.021146,0.141848
4,4,0.0,8.0,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.463496,0.099763,0.194714,0.067542,0.0911

### train
- 行数: 257117

In [8]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 230)

### test
- 行数: 521650

In [9]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 230)

In [10]:
del all_pitch
gc.collect()

60

In [11]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [12]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

## Dataset作成

In [13]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [14]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 6.9923570049658075, 
        'lambda_l2': 0.002378623984798833, 
        'num_leaves': 18, 
        'feature_fraction': 0.45199999999999996, 
        'bagging_fraction': 0.9799724836460725, 
        'bagging_freq': 4, 
        'min_child_samples': 20
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 1.13785 + 0.0022687
[200]	cv_agg's multi_logloss: 1.11781 + 0.00247314
[300]	cv_agg's multi_logloss: 1.11161 + 0.002505
[400]	cv_agg's multi_logloss: 1.10781 + 0.0025639
[500]	cv_agg's multi_logloss: 1.1055 + 0.00270729
[600]	cv_agg's multi_logloss: 1.10364 + 0.00285791
[700]	cv_agg's multi_logloss: 1.10252 + 0.00277696
[800]	cv_agg's multi_logloss: 1.10162 + 0.00288271
[900]	cv_agg's multi_logloss: 1.10107 + 0.00293321
[1000]	cv_agg's multi_logloss: 1.1007 + 0.00297548
[1100]	cv_agg's multi_logloss: 1.10049 + 0.00297465
[1200]	cv_agg's multi_logloss: 1.10044 + 0.00304715
[1300]	cv_agg's multi_logloss: 1.10048 + 0.00311972
Best num_boost_round: 1204
Best CV score: 1.1004215512272495
CPU times: user 2h 3min 54s, sys: 21.2 s, total: 2h 4min 16s
Wall time: 15min 34s


In [15]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [16]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 39min 45s, sys: 6.72 s, total: 39min 52s
Wall time: 4min 59s


### Feature Importance

In [17]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
208,rank_x_year_dif_b-c,1766
167,batter_cnt,1845
193,bmi_dif_p-b,1868
216,predict_2,1938
56,elapsed_min,1977
209,bmi_dif_b-c,1980
0,pitch_cnt_in_game,1987
222,predict_8,2035
221,predict_7,2136
215,predict_1,2145


In [18]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 56s, sys: 1.68 s, total: 6min 58s
Wall time: 55.2 s


## 結果出力

In [19]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.440722,0.047816,0.130129,0.316020,0.002838,0.000016,0.000018,0.062442
1,1,0.479414,0.090333,0.116752,0.262779,0.005222,0.000016,0.000010,0.045474
2,2,0.344223,0.103355,0.181227,0.133855,0.153278,0.000091,0.000018,0.083953
3,3,0.409261,0.120297,0.152470,0.243084,0.022606,0.000042,0.000022,0.052219
4,4,0.203263,0.052717,0.354480,0.289769,0.014182,0.000033,0.000022,0.085533


In [20]:
submit.to_csv(SUBMIT, header=False, index=False)

### コース予測で使用

In [21]:
%%time
if not use_sub_model:
    train_predict = lgb_model.predict(train_d, num_iteration = lgb_model.best_iteration)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


In [22]:
if not use_sub_model:
    df_train_predict = pd.DataFrame(train_predict).reset_index()
    submodel = pd.concat([df_train_predict, submit], ignore_index=True)
    submodel.drop(columns=['index'], inplace=True)
    submodel.rename(columns={
        0: 'predict_straight',
        1: 'predict_curve',
        2: 'predict_slider',
        3: 'predict_shoot',
        4: 'predict_fork',
        5: 'predict_changeup',
        6: 'predict_sinker',
        7: 'predict_cutball'
    }, inplace=True)
    print(submodel.shape)
    submodel.head()

In [23]:
if not use_sub_model:
    submodel.to_feather(OUT_SUBMODEL)

### SIGNATE

In [24]:
'signate submit --competition-id=275 ./{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=275 ./submit/ball_18_sub.csv --note feat=230_cv=1.1004215512272495'

### GCS

In [25]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ./submit/ball_18_sub.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 18 (217) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1102
- Best CV score: 1.1004234715781385
- Wall time: 13min 42s
- 評価結果  : 1.3954167

#### 17 (191) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1034107856339535
- Wall time: 26min 44s
- 評価結果  : 1.3963057

#### 16 (181) ... 選手IDごとの2017年のコース実績
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1178
- Best CV score: 1.1021930539837317
- Wall time: 12min 35s
- 評価結果  : 1.3993836

#### 15 (153) ... 左右で球種/コースを別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1189
- Best CV score: 1.1013869435441306
- Wall time: 11min 9s
- 評価結果  : 未提出

#### 14 (153) ... 左右で球種を別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1240
- Best CV score: 1.1021952294615331
- Wall time: 11min 34s
- 評価結果  : 1.3981900

#### 13 (157) ... 前の投球からの時間差を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1416
- Best CV score: 1.1062463941695753
- Wall time: 13min 10s
- 評価結果  : 1.4012184

#### 12 (156) ... コースの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1353
- Best CV score: 1.106942086205103
- Wall time: 27min 41s
- 評価結果  : 1.4016077

#### 11 (143) ... ボールカウントの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1209
- Best CV score: 1.1075153721728936
- Wall time: 23min 19s
- 評価結果  : 未提出

#### 10_tune (135) ... 再チューニング
- 'num_leaves' : 15,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1260
- Best CV score: 1.1118897437428692
- Wall time: 8min 40s
- 評価結果  : 1.4060397

#### 10s (135) ... trainを75%サンプリング
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1412
- Best CV score: 1.1190287120710383
- Wall time: 12min 5s
- 評価結果  : 1.4310869

#### 10 (135) ... 5相当に戻した
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1112
- Best CV score: 1.111265000190272
- Wall time: 9min 30s
- 評価結果  : 1.4008712

#### 9 (136) ... 5相当に戻した=>NG
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1106
- Best CV score: 1.1113177029272225
- Wall time: 25min 44s (local)
- 評価結果  : 1.5133521

#### 8 (157)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1093728644236824
- Wall time: 10min 4s
- 評価結果  : 1.4925832

#### 7 (235)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.108590082437519
- Wall time: 12min 56s
- 評価結果  : 1.4946937

#### 6 (207)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1135
- Best CV score: 1.1086837935965734
- Wall time: 11min 54s
- 評価結果  : 1.3999194

#### 5-tune (参考)
- 'num_leaves' : 18,
- 'learning_rate' : 0.05,
- Best num_boost_round: 2395
- Best CV score: 1.1103399384119979
- Wall time: 18min 29s (n1-standard-8)

#### 5-tune (135)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- CV score: 1.1112402566616455
- 評価結果  : 1.3991780
- CV Wall time: 27min 48s (local) -> 18min 24s (n1-standard-4) -> 9min 3s (n1-standard-8)

#### 5 (135)
- 'num_leaves' : 31,
- 'learning_rate' : 0.1,
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806